# Implementation of paSST feature extractor on .wav files from FreeSound

In [ ]:
%pip install hear21passt

In [19]:
%pip install librosa

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 101.4 MB/s eta 0:00:00m eta 0:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9/9 [librosa]5;237m━━━━ 8/9 [librosa]]
Note: you may need to restart the kernel to use updated packages.


In [56]:
import librosa
import numpy as np
import torch
from hear21passt.base import get_basic_model,get_model_passt

In [59]:
def load_real_audio(file_path, target_sr=32000, max_len_seconds=10):
    waveform, sr = librosa.load(file_path, sr=target_sr, mono=True)

    max_len_samples = target_sr * max_len_seconds
    if len(waveform) < max_len_samples:
        pad_len = max_len_samples - len(waveform)
        waveform = np.pad(waveform, (0, pad_len))
    else:
        waveform = waveform[:max_len_samples]

    return torch.tensor(waveform, dtype=torch.float32)

In [60]:
model = get_basic_model(mode="embed_only")
model.eval()
model = model.cuda()



 Loading PASST TRAINED ON AUDISET 


PaSST(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(1, 768, kernel_size=(16, 16), stride=(10, 10))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): LayerNorm((768,), ep

In [61]:
with torch.no_grad():
    waveform = load_real_audio("datasets/audio_files/33711__acclivity__excessiveexposure.wav")
    audio = waveform.unsqueeze(0)
    audio_wave = audio.cuda()
    with torch.no_grad():
        embedding = model(audio_wave)
    print("Embedding Shape:",embedding.shape)

Embedding Shape: torch.Size([1, 768])


In [62]:
with torch.no_grad():
    waveform = load_real_audio("datasets/audio_files/36105__erh__roswell.wav")
    audio = waveform.unsqueeze(0)
    audio_wave = audio.cuda()
    with torch.no_grad():
        embedding = model(audio_wave) 
    print("Embedding Shape:",embedding.shape)

Embedding Shape: torch.Size([1, 768])


In [63]:
with torch.no_grad():
    waveform = load_real_audio("datasets/audio_files/222993__zyrytsounds__people-talking.wav")
    audio = waveform.unsqueeze(0)
    audio_wave = audio.cuda()
    with torch.no_grad():
        embedding = model(audio_wave)
    print("Embedding Shape:",embedding.shape)

Embedding Shape: torch.Size([1, 768])


In [64]:
filepaths = [
    "datasets/audio_files/33711__acclivity__excessiveexposure.wav",
    "datasets/audio_files/222993__zyrytsounds__people-talking.wav",
    "datasets/audio_files/36105__erh__roswell.wav" 
]

batch_waveforms = [load_real_audio(fp) for fp in filepaths]
batch_audio = torch.stack(batch_waveforms).cuda()

with torch.no_grad():
    embeddings = model(batch_audio)

print("Batch Embedding Shape:", embeddings.shape)

Batch Embedding Shape: torch.Size([3, 768])
